# Исследование поведения клиентов оператора

Заказчик - федеральный оператор мобильной связи «Мегалайн». Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Даны данные о поведении клиентов, которые уже перешли на эти тарифы. Предобработка данных проведена на предыдущем этапе.

Нужно построить модель для задачи классификации, которая выберет подходящий тариф для каждого из клиентов. Ключевая метрика качества работы модели - accuracy. Целевое значение accuracy - 0,75.

__Описание данных:__


- `сalls` — количество звонков;
- `minutes` — суммарная длительность звонков в минутах;
- `messages` — количество sms-сообщений;
- `mb_used` — израсходованный интернет-трафик в Мб;
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

__Цель исследования:__ Построить модель для задачи классификации клиентов по тарифам, с наилучшим показателем точности.


__Ход исследования:__
 1. Обзор данных:
 2. Подготовка данных к обучению моделей.
 3. Обучение моделей.
 4. Оценка качества моделей.

### Подготовка окружения
Загрузим все библиотеки, требуемые для работы в ходе проекта в рамках одной ячейки.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

---

## Открытие файла с данными и изучение информации о данных

In [2]:
try:
    users_behavior = pd.read_csv('/datasets/users_behavior.csv')
except FileNotFoundError:
    users_behavior = pd.read_csv('-') # Прямая ссылка на датасет удалена в связи с правилами публикации проектов

Изучим информацию о данных. Выведем на экран первые пять строк таблицы, основную обзорную и статистическую информацию.

In [3]:
display(users_behavior.head())
print(users_behavior.info())
users_behavior.describe()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В датасете 3214 наблюдений и 4 переменных. Целевой признак - подходящий абоненту тариф. Как видно из среднего числа целевого признака - среди абонентов 30,7% - пользователи тарифа "Ультра", 69,3% - пользователи тарифа "Смарт".

## Подготовка данных

Разделим данные на обучающую, валидационную и тестовую выборку. Тестовую и валидационную выборки возьмем одинакового размера - по 20% от датасете, тренировочную - 60% от датасета.

In [4]:
features = users_behavior.drop('is_ultra', axis=1)
target = users_behavior['is_ultra']

In [5]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.2, random_state=12345)

In [6]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=.25, random_state=12345)

In [7]:
print('Тренировочная выборка, размерность признаков', features_train.shape)
print('Валидационная выборка, размерность признаков', features_valid.shape)
print('Тестовая выборка, размерность признаков', features_test.shape)

Тренировочная выборка, размерность признаков (1928, 4)
Валидационная выборка, размерность признаков (643, 4)
Тестовая выборка, размерность признаков (643, 4)


После разделения можно приступить к построению моделей.

## Обучение моделей

В рамках исследования построим четыре основных модели классификации и оптимизируем их гиперпараметры. Построим следующие модели:

1. Логистическая регрессия.
2. Дерево решений.
3. Случайный лес.
4. Метод ближайших соседей.

### Логистическая регрессия

Обучим модель логистической регрессии на тренировочном датасете.

In [8]:
logit = LogisticRegression(random_state=12345)
logit.fit(features_train, target_train)

predictions_train = logit.predict(features_train)
accuracy_logit_train = accuracy_score(target_train, predictions_train)
predictions_valid = logit.predict(features_valid)
accuracy_logit_valid = accuracy_score(target_valid, predictions_valid)

print('Accuracy лучшей модели на тренировочной выборке составил', accuracy_logit_train,
     '\nAccuracy лучшей модели на валидационной выборке составил', accuracy_logit_valid)

Accuracy лучшей модели на тренировочной выборке составил 0.703838174273859 
Accuracy лучшей модели на валидационной выборке составил 0.6967340590979783


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модель не склонна к переобучению, следовательно предсказания на тренировочной и валидационной выборке различаются не слишком сильно.

### Дерево решений

Подберем оптимальные параметры для обучения модели дерева решений и выберем лучшую модель.

In [9]:
best_tree_model = None
best_result = 0
best_tree_depth = 0
for depth in range(1, 20):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_train, target_train)
    predictions_valid = tree.predict(features_valid)
    accuracy_tree_valid = accuracy_score(target_valid, predictions_valid)
    if accuracy_tree_valid > best_result:
        best_tree_model = tree
        best_result = accuracy_tree_valid
        best_tree_depth = depth
predictions_train = tree.predict(features_train)
accuracy_tree_train = accuracy_score(target_train, predictions_train)
predictions_valid = tree.predict(features_valid)
accuracy_tree_valid = accuracy_score(target_valid, predictions_valid)

print('Accuracy модели на тренировочной выборке составил', accuracy_tree_train,
     '\nAccuracy модели на валидационной выборке составил', accuracy_tree_valid,
     '\nГлубина дерева', best_tree_depth)

Accuracy модели на тренировочной выборке составил 0.9704356846473029 
Accuracy модели на валидационной выборке составил 0.7356143079315708 
Глубина дерева 7


In [10]:
best_tree_model = None
best_result = 0
best_tree_depth = 0

for depth in range(1, 20):
    tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree.fit(features_train, target_train)
    predictions_valid = tree.predict(features_valid)
    accuracy_tree_valid = accuracy_score(target_valid, predictions_valid)
    if accuracy_tree_valid > best_result:
        best_tree_model = tree
        best_result = accuracy_tree_valid
        best_tree_depth = depth
        
predictions_train = best_tree_model.predict(features_train)
accuracy_tree_train = accuracy_score(target_train, predictions_train)
predictions_valid = best_tree_model.predict(features_valid)
accuracy_tree_valid = accuracy_score(target_valid, predictions_valid)

print('Accuracy модели на тренировочной выборке составил', accuracy_tree_train,
     '\nAccuracy модели на валидационной выборке составил', accuracy_tree_valid,
     '\nГлубина дерева', best_tree_depth)

Accuracy модели на тренировочной выборке составил 0.8506224066390041 
Accuracy модели на валидационной выборке составил 0.7744945567651633 
Глубина дерева 7


Модель склонна к переобучению, за счет чего показывает лучшие результаты на тренировочной выборке, чем на валидационной.

### Случайный лес

Таким же образом обучим модель случайного леса и выберем лучшие параметры для модели.

In [11]:
best_forest_model = None
best_result = 0
best_forest_depth = 0
best_forest_est = 0

for depth in range(1, 20):
    for est in range(1, 101, 5):
        forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        forest.fit(features_train, target_train)
        predictions_valid = forest.predict(features_valid)
        accuracy_forest_valid = accuracy_score(target_valid, predictions_valid)
        if accuracy_forest_valid > best_result:
            best_forest_model = forest
            best_result = accuracy_forest_valid
            best_forest_depth = depth
            best_forest_est = est
            
predictions_train = best_forest_model.predict(features_train)
accuracy_forest_train = accuracy_score(target_train, predictions_train)
predictions_valid = best_forest_model.predict(features_valid)
accuracy_forest_valid = accuracy_score(target_valid, predictions_valid)

print('Accuracy модели на тренировочной выборке составил', accuracy_forest_train,
     '\nAccuracy модели на валидационной выборке составил', accuracy_forest_valid,
     '\nГлубина деревьев', best_forest_depth,
     '\nКоличество деревьев', best_forest_est)

Accuracy модели на тренировочной выборке составил 0.9133817427385892 
Accuracy модели на валидационной выборке составил 0.8009331259720062 
Глубина деревьев 12 
Количество деревьев 76


Модель еще более склонна к переобучению, но показывает лучший результат на валидационной выборке.

### Метод ближайших соседей

In [12]:
best_neighbors_model = None
best_result = 0
best_n_neighbors = 0

for n_neighbors in range(1, 50):
    neighbors = KNeighborsClassifier(n_neighbors=n_neighbors)
    neighbors.fit(features_train, target_train)
    predictions_valid = neighbors.predict(features_valid)
    accuracy_neighbors_valid = accuracy_score(target_valid, predictions_valid)
    if accuracy_neighbors_valid > best_result:
        best_neighbors_model = forest
        best_result = accuracy_neighbors_valid
        best_n_neighbors = n_neighbors
        
predictions_train = best_neighbors_model.predict(features_train)
accuracy_neighbors_train = accuracy_score(target_train, predictions_train)
predictions_valid = best_neighbors_model.predict(features_valid)
accuracy_neighbors_valid = accuracy_score(target_valid, predictions_valid)

print('Accuracy модели на тренировочной выборке составил', accuracy_neighbors_train,
     '\nAccuracy модели на валидационной выборке составил', accuracy_neighbors_valid,
     '\nКоличество соседей', best_n_neighbors)

Accuracy модели на тренировочной выборке составил 0.9870331950207469 
Accuracy модели на валидационной выборке составил 0.7853810264385692 
Количество соседей 12


Метод ближайших соседей еще более склонен к переобучению, чем все предыдущие модели.

## Оценка качества моделей

Оценим качество моделей на тестовом датасете и  сравним все три показателя - точность предсказания на тренировочной, валидационной и тестовой выборке. Также добавим в сравнение результаты dummy модели. Так как у нас ~30% - абоненты тарифа "Ультра", остальные - абоненты тарифа "Смарт", это модель будет выбирать тариф "Ультра" для новых данных с вероятностью 0,3.

In [13]:
dummy = DummyClassifier(random_state=12345, strategy='most_frequent')
dummy.fit(features_train, target_train)

models = (logit, best_tree_model, best_forest_model, best_neighbors_model, dummy)
names_of_models = ('Logistic regression', 'Desision tree', 'Random Forest', 'K-neighbors', 'Dummy')
comparison = {}

for i in range(len(models)):
    model = models[i]
    train_accuracy = accuracy_score(target_train, model.predict(features_train))
    valid_accuracy = accuracy_score(target_valid, model.predict(features_valid))
    test_accuracy = accuracy_score(target_test, model.predict(features_test))
    comparison[names_of_models[i]] = (train_accuracy, valid_accuracy, test_accuracy)

comparison = pd.DataFrame.from_dict(comparison).T
comparison.columns = ('Train accuracy', 'Valid accuracy', 'Test accuracy')

comparison

,Train accuracy,Valid accuracy,Test accuracy
Logistic regression,0.703838,0.696734,0.702955
Desision tree,0.850622,0.774495,0.788491
Random Forest,0.913382,0.800933,0.797823
K-neighbors,0.987033,0.785381,0.785381
Dummy,0.694502,0.688958,0.695179


Все модели показали лучший результат, чем сравнительная модель Dummy. Она выбирала класс в соответствии с распределением исходных классов в тренировочной выборке. Этот результат чуть выше, чем у полностью случайного назначения, которое вероятнее всего показало бы результат близкий к 0,5.

Лучший результат среди полноценных моделей на тестовой выборке показала модель случайного леса. Несмотря на явную тененцию к переобучению, модель работает наилучшим образом, за ней следуют с минимальным отрывом модели дерева решений и ближайших соседей, также показывающие значительно более высокие результаты на тренировочной выборке.

## Общий вывод

Для компании «Мегалайн» целесообразнее всего использовать метод случайных деревьев. С вероятностью ~ 80% он позволяет предсказать, какой тариф оптимальнее всего подходит для клиента, в зависимости от его характера потребления трафика, сообщений и звонков.

Количество деревьев в оптимальной модели - 76, глубина каждого дерева - 12.

Дальнейшие исследования лучше сосредоточить именно на этой модели - глубже исследовать гиперпараметры и поведение модели на новых данных. Остальные модели также не стоит отбрасывать совсем -  при увеличении количества данных метод ближайших соседей модет начать работать все лучше и лучше, так как он наиболее оптимален на больших датасетах.

Кроме этого компании следует обратить внимание на удовлетворенность качеством обслуживания клиентов. Не исключено, что при использовании того или иного тарифа клиент может быть недоволен им и он ему может не подходить. Таким образом предсказанный оптимальный тариф может также не подходить клиенту, переходящему со старого. Собранные данные удовлетворенности клиентов позволят предсказать не оптимальный тариф, а то, насколько конкретный клиент, переходящий на него, будет доволен сменой тарифа. Так будет возможно предлагать индивидуальные услуги и скидки клиентам, что бы оптимизировать выручку безболезненно для клиента.